In [1]:
import gspread
import os
import pandas as pd
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request

In [2]:
signos = ['aries','touro','gemeos','cancer','leao','virgem','libra','escorpiao','sagitario','capricornio','aquario','peixes']

In [3]:
url = 'https://joaobidu.com.br/horoscopo/signos/previsao-'

In [4]:
url_imagem = 'https://joaobidu.com.br/static/img/'

In [5]:
lista_texto = {}
lista_imagem = {}
for signo in signos:
    resposta = Request(url+signo,headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(resposta,timeout=10).read()
    conteudo = BeautifulSoup(html, 'html.parser')
    imagem = conteudo.p.img.get('src').replace('../static/img/','')
    texto = conteudo.p.get_text()
    lista_texto[signo] = texto.replace('\n','')
    lista_imagem[signo] = url_imagem+imagem

In [6]:
df_signos_descricao = pd.DataFrame(list(lista_texto.items()),
                   columns=['Signos', 'Descrição'])

In [7]:
df_signos_imagens = pd.DataFrame(list(lista_imagem.items()),
                   columns=['Signos', 'Imagem'])

In [8]:
df_merge_signos = pd.merge(df_signos_descricao,df_signos_imagens, how='right', on = 'Signos')

In [9]:
df_merge_signos

,Signos,Descrição,Imagem
0,aries,"SEX - A semana está acabando e, com a Lua de m...",https://joaobidu.com.br/static/img/ico-aries.png
1,touro,SEX - Com a entrada da Lua em Câncer nesta mad...,https://joaobidu.com.br/static/img/ico-touro.png
2,gemeos,"SEX - Sextou, meu bem, e com a Lua brilhando l...",https://joaobidu.com.br/static/img/ico-gemeos.png
3,cancer,SEX - A Lua brilha em seu signo a partir de ho...,https://joaobidu.com.br/static/img/ico-cancer.png
4,leao,"SEX - Nessa madrugada, a Lua se muda para seu ...",https://joaobidu.com.br/static/img/ico-leao.png
5,virgem,"SEX - Sextou, virginiane, e o seu lado sonhado...",https://joaobidu.com.br/static/img/ico-virgem.png
6,libra,"SEX - A gloriosa sexta chegou, mas a entrada d...",https://joaobidu.com.br/static/img/ico-libra.png
7,escorpiao,SEX - Com a Lua brilhando em Câncer a partir d...,https://joaobidu.com.br/static/img/ico-escorpi...
8,sagitario,SEX - A Lua está indicando um período de trans...,https://joaobidu.com.br/static/img/ico-sagitar...
9,capricornio,"SEX - Sextou, bebê, e com a Lua na sua Casa do...",https://joaobidu.com.br/static/img/ico-caprico...


In [10]:
scopes = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"]

In [11]:
json_file = 'chave/' + str(os.listdir('chave/')[0])

In [12]:
credentials = service_account.Credentials.from_service_account_file(json_file)
scoped_credentials = credentials.with_scopes(scopes)
gc = gspread.authorize(scoped_credentials)

In [13]:
plan_aux = gc.open("BotTelegram")
aba_aux = plan_aux.worksheet("Teste")

In [14]:
aba_aux.update([df_merge_signos.columns.values.tolist()] + df_merge_signos.values.tolist())

{'spreadsheetId': '1BAtnINF33KdEYHbVyuJ7tKYi4k94ZnaX51yIwaO4XC4',
 'updatedRange': 'Teste!A1:C13',
 'updatedRows': 13,
 'updatedColumns': 3,
 'updatedCells': 39}